In [1]:
# This notebook includes model developement with target variable as Satisfaction_with_Remote_Work

In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

In [3]:
file_path = 'Data/Cleaned_Impact_of_Remote_Work_on_Mental_Health.csv'
df = pd.read_csv(file_path)

#### Data cleanup for unrelated values for Satisfaction_with_Remote_Work

In [4]:
df = df.loc[df['Work_Location'] != 'Onsite']

#### Remove outliers

In [5]:
df = df.drop(df[(df['Satisfaction_with_Remote_Work'] == 'Unsatisfied') & (df['Work_Life_Balance_Rating'] == 5)].index)

#### Encoding

In [6]:
df['Sleep_Quality_Encoded'] = df['Sleep_Quality'].apply(lambda x: 2 if x == 'Good' else (1 if x == 'Average' else 0))

In [7]:
df['Stress_Level_Encoded'] = df['Stress_Level'].apply(lambda x: 2 if x == 'Low' else (1 if x == 'Medium' else 0))

In [8]:
def classify_satifaction(row):
    if row['Satisfaction_with_Remote_Work'] == 'Satisfied':
        return 1
    elif row['Satisfaction_with_Remote_Work'] == 'Neutral' and row['Work_Life_Balance_Rating'] >= 3:
        return 1
    else:
        return 0

df['Satisfaction_with_Remote_Work_Encoded'] = df.apply(classify_satifaction, axis = 1)

In [9]:
features_to_drop = ['Employee_ID', 'Satisfaction_with_Remote_Work', 'Satisfaction_with_Remote_Work_Encoded', 'Sleep_Quality', 
                    'Stress_Level', 'Work_Location']

In [10]:
X = pd.get_dummies(df.drop(features_to_drop, axis = 1), drop_first = True)
y = df['Satisfaction_with_Remote_Work_Encoded']

In [11]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

In [12]:
class_distribution = df['Satisfaction_with_Remote_Work_Encoded'].value_counts()
print(class_distribution)

Satisfaction_with_Remote_Work_Encoded
1    1739
0    1390
Name: count, dtype: int64


In [13]:
scale_pos_weight = class_distribution.max() / class_distribution.min()
scale_pos_weight

np.float64(1.2510791366906475)

## Using RandomForest

#### Tuning

In [14]:
from bayes_opt import BayesianOptimization
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold
from sklearn.ensemble import RandomForestClassifier
import numpy as np

def RandomForest_Bayesian(n_estimators, max_depth, min_samples_split, min_samples_leaf):
    model = RandomForestClassifier(
        max_depth = int(max_depth),
        n_estimators = int(n_estimators),
        min_samples_split = int(min_samples_split),
        min_samples_leaf = int(min_samples_leaf),
        class_weight = 'balanced',
        random_state = 42
    )
    
    # Performing cross-validation
    cv = StratifiedKFold(n_splits = 5, shuffle = True, random_state = 42)
    scores = cross_val_score(model, X_train, y_train, cv = cv, scoring = 'precision')
    return np.mean(scores)

param_bounds = {
    'n_estimators': (50, 300),
    'max_depth': (3, 20),
    'min_samples_split': (2, 10),
    'min_samples_leaf': (1, 5)
}

optimizer = BayesianOptimization(
    f = RandomForest_Bayesian,
    pbounds = param_bounds,
    random_state = 42,
    verbose = 2
)

optimizer.maximize(init_points = 20, n_iter = 50)
print("Best Parameters:", optimizer.max)

|   iter    |  target   | max_depth | min_sa... | min_sa... | n_esti... |
-------------------------------------------------------------------------
| 1         | 0.7247    | 9.367     | 4.803     | 7.856     | 199.7     |
| 2         | 0.7253    | 5.652     | 1.624     | 2.465     | 266.5     |
| 3         | 0.7199    | 13.22     | 3.832     | 2.165     | 292.5     |
| 4         | 0.7027    | 17.15     | 1.849     | 3.455     | 95.85     |
| 5         | 0.7276    | 8.172     | 3.099     | 5.456     | 122.8     |
| 6         | 0.7151    | 13.4      | 1.558     | 4.337     | 141.6     |
| 7         | 0.7262    | 10.75     | 4.141     | 3.597     | 178.6     |
| 8         | 0.7165    | 13.07     | 1.186     | 6.86      | 92.63     |
| 9         | 0.7256    | 4.106     | 4.796     | 9.725     | 252.1     |
| 10        | 0.7261    | 8.178     | 1.391     | 7.474     | 160.0     |
| 11        | 0.725     | 5.075     | 2.981     | 2.275     | 277.3     |
| 12        | 0.7265    | 7.399     | 

#### Train

In [15]:
rf_model = RandomForestClassifier(
    n_estimators = int(optimizer.max['params']['n_estimators']),
    max_depth = int(optimizer.max['params']['max_depth']),
    min_samples_split = int(optimizer.max['params']['min_samples_split']),
    min_samples_leaf = int(optimizer.max['params']['min_samples_leaf']),
    class_weight = 'balanced',
    random_state = 42
)

rf_model.fit(X_train, y_train)

RandomForestClassifier(class_weight='balanced', max_depth=3, min_samples_leaf=3,
                       min_samples_split=8, n_estimators=50, random_state=42)

#### Predict

In [17]:
y_pred = rf_model.predict(X_test)

print("\nClassification Report:\n", classification_report(y_test, y_pred))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))


Classification Report:
               precision    recall  f1-score   support

           0       0.67      0.66      0.66       273
           1       0.74      0.75      0.75       353

    accuracy                           0.71       626
   macro avg       0.71      0.70      0.71       626
weighted avg       0.71      0.71      0.71       626

Confusion Matrix:
 [[179  94]
 [ 87 266]]
